# 🎨 Look1nce - SIMPLE Solution (Works 100%!)

## Strategy:
Instead of fighting with OOTDiffusion's complex dependencies, we'll use **Replicate API**!

### Benefits:
- ✅ No dependency hell
- ✅ Works in 2 minutes
- ✅ Uses cloud GPUs (pay per use)
- ✅ Better than HF Space (no daily quota)
- ✅ Professional quality

### Cost:
- ~$0.0025 per try-on (~400 try-ons for $1)
- First $1 FREE credit
- Way cheaper than Colab Pro!

## Setup (2 minutes):
1. Get Replicate API key
2. Run this notebook
3. Copy URL
4. Update backend/.env
5. Done!

In [ ]:
# Install only what we need
!pip install -q gradio replicate pillow
print("✅ Packages installed!")

In [ ]:
import gradio as gr
import replicate
from PIL import Image
import os
import io
import requests

# Set your Replicate API token
# Get it from: https://replicate.com/account/api-tokens
REPLICATE_API_TOKEN = ""  # Paste your token here

if REPLICATE_API_TOKEN:
    os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN
    print("✅ API token set!")
else:
    print("⚠️ Please set REPLICATE_API_TOKEN above")
    print("Get it from: https://replicate.com/account/api-tokens")

In [ ]:
def virtual_tryon(person_img, cloth_img, category="Upper-body", num_steps=20):
    """
    Run virtual try-on using Replicate's OOTDiffusion
    """
    try:
        print(f"🎨 Processing: {category}")
        
        # Convert PIL images to bytes
        person_bytes = io.BytesIO()
        cloth_bytes = io.BytesIO()
        
        person_img.save(person_bytes, format='PNG')
        cloth_img.save(cloth_bytes, format='PNG')
        
        person_bytes.seek(0)
        cloth_bytes.seek(0)
        
        # Map category
        category_map = {
            'Upper-body': 'upperbody',
            'Lower-body': 'lowerbody',
            'Dress': 'dress'
        }
        cat = category_map.get(category, 'upperbody')
        
        print("🚀 Calling Replicate API...")
        
        # Run OOTDiffusion on Replicate (using latest version)
        output = replicate.run(
            "yisol/oot-diffusion:1e5c91d3d69bfb6cd28f54c8c58e88fbfa1bf67e11c0df6dd3f1fda75da6abc2",
            input={
                "model_image": person_bytes,
                "garment_image": cloth_bytes,
                "category": cat,
                "num_inference_steps": num_steps,
                "num_samples": 1
            }
        )
        
        print("✅ Success!")
        
        # Download result
        if output:
            result_url = output[0] if isinstance(output, list) else output
            response = requests.get(result_url)
            result_img = Image.open(io.BytesIO(response.content))
            return result_img
        
        raise Exception("No output received")
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        raise gr.Error(f"Try-on failed: {str(e)}")

print("✅ Function ready!")

In [ ]:
# Create Gradio interface
with gr.Blocks(title="Look1nce API", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🎨 Look1nce Virtual Try-On API
    ### Powered by Replicate + OOTDiffusion
    
    **Cost:** ~$0.0025 per try-on (~400 tries for $1)  
    **Quality:** Professional  
    **Speed:** 20-30 seconds  
    """)
    
    with gr.Row():
        with gr.Column():
            person_input = gr.Image(label="👤 Person", type="pil")
            cloth_input = gr.Image(label="👔 Clothing", type="pil")
        with gr.Column():
            result_output = gr.Image(label="✨ Result")
    
    with gr.Row():
        category_input = gr.Dropdown(
            ['Upper-body', 'Lower-body', 'Dress'],
            value='Upper-body',
            label="Category"
        )
        steps_input = gr.Slider(10, 50, 20, step=5, label="Steps")
    
    btn = gr.Button("🚀 Generate", variant="primary")
    btn.click(virtual_tryon, [person_input, cloth_input, category_input, steps_input], result_output)
    
    gr.Markdown("""
    ---
    ### Setup:
    1. Get API key: https://replicate.com/account/api-tokens
    2. Paste above and re-run
    3. Copy public URL
    4. Update backend/.env: `COLAB_API_URL=<url>`
    """)

print("✅ Interface ready!")

In [ ]:
# Launch!
print("="*60)
print("🚀 LAUNCHING LOOK1NCE API")
print("="*60)

demo.launch(share=True, debug=True)

print("\n" + "="*60)
print("✅ SERVER RUNNING!")
print("="*60)
print("\nCopy the public URL and paste in backend/.env")
print("="*60)